In [ ]:
import dropbox
from u import *

dbx = dropbox.Dropbox(os.environ['DROPBOX_TOKEN'])
dbx.users_get_current_account()

def upload(src_path, dst_path):
    print('Uploading file', src_path, 'to', dst_path)
    f = open(src_path, 'rb')
    file_size = os.path.getsize(src_path)

    CHUNK_SIZE = 4 * 1024 * 1024

    if file_size <= CHUNK_SIZE:
        print(dbx.files_upload(f.read(), dst_path))
    else:
        upload_session_start_result = dbx.files_upload_session_start(f.read(CHUNK_SIZE))
        cursor = dropbox.files.UploadSessionCursor(session_id=upload_session_start_result.session_id,
                                                   offset=f.tell())
        commit = dropbox.files.CommitInfo(path=dst_path)

        while f.tell() < file_size:
            if ((file_size - f.tell()) <= CHUNK_SIZE):
                print(dbx.files_upload_session_finish(f.read(CHUNK_SIZE), cursor, commit))
            else:
                dbx.files_upload_session_append(f.read(CHUNK_SIZE),
                                                cursor.session_id,
                                                cursor.offset)
                cursor.offset = f.tell()

In [ ]:
for exp in Res.ls()[0]:
    c = Config(exp)
    max_step = max(c.get_saved_model_steps())
    path = c.model_save(max_step).rel('.')
    upload(path, '/' + path)